In [ ]:
!pip install ortools
!pip install pandas

In [ ]:
from ortools.linear_solver import pywraplp

In [ ]:
def flag_calculate(bins_df):
    bin_capacities = bins_df['base'].to_list()
    bin_cap_max = list()
    bin_cap_min  = list()
    
    # データの初期処理
    for i, flg in enumerate(bins_df.loc[:,"flag"]):
        if flg == 'm':
            bin_cap_max.append(bin_capacities[i] + 10000)
            bin_cap_min.append(bin_capacities[i] - 10000)
        elif flg == 'i':
            if bin_capacities[i] > 500000:
                bin_cap_max.append(bin_capacities[i] + 50000)
                bin_cap_min.append(bin_capacities[i] - 50000)
            else:
                bin_cap_max.append(bin_capacities[i] + 10000)
                bin_cap_min.append(bin_capacities[i] - 10000)
        else:
            raise ValueError("フラグの値が不正です")
    
    bins_df['max'] = bin_cap_max
    bins_df['min'] = bin_cap_min

    return bins_df

In [ ]:
def make_data(bins_df:list, weights:list, values:list)->dict:

    bin_capacities = bins_df['max'].to_list()
    bin_minimumreq = bins_df['min'].to_list()
    bin_classes = bins_df['class2'].to_list()
    
    data = {}
    data['weights'] = weights
    data['values'] = values
    assert len(data['weights']) == len(data['values'])
    data['num_items'] = len(data['weights'])
    data['all_items'] = range(data['num_items'])
    data['classes'] = bin_classes
    data['bin_capacities'] = bin_capacities
    data['num_bins'] = len(data['bin_capacities'])
    data['all_bins'] = range(data['num_bins'])
    return data, bin_minimumreq

In [ ]:
def slv_multiknap(data,bin_minimumreq):
    solver = pywraplp.Solver.CreateSolver('SCIP')
    if solver is None:
        print('SCIP solver unavailable.')

    x = {}
    for i in data['all_items']:
        for b in data['all_bins']:
            x[i, b] = solver.BoolVar(f'x_{i}_{b}')

    # それぞれのアイテムはバッグに1つ以上入らない
    for i in data['all_items']:
        solver.Add(sum(x[i, b] for b in data['all_bins']) <= 1)

    # それぞれのバッグにはアイテムが１つ以上入る
    for b in data['all_bins']:
        solver.Add(sum(x[i, b] for i in data['all_bins']) >= 1)

    # キャパシティの上限・下限の設定
    for b in data['all_bins']:
        # 上限容量
        solver.Add(
            sum(x[i, b] * data['weights'][i]
                for i in data['all_items']) <= data['bin_capacities'][b])
        # 下限容量
        solver.Add(
            sum(x[i, b] * data['weights'][i]
                for i in data['all_items']) >= bin_minimumreq[b])

        # 個数の最大化を目標にする
        objective = solver.Objective()
        for i in data['all_items']:
            for b in data['all_bins']:
                objective.SetCoefficient(x[i, b], data['values'][i])

        objective.SetMaximization()
    
    status = solver.Solve()

    res = list()

    if status == pywraplp.Solver.OPTIMAL:
        """デバッグ用
        # print(f'最大個数: {objective.Value()}\n') 
        # txt = f'  最大個数: {objective.Value()}\n'
        # txt = ''  
        """

        total_weight = 0
        for b in data['all_bins']:
            """デバッグ用
            print(f'  Bin {b}')
            txt += f'  Bin {b}'  
            """
            bin_weight = 0
            bin_value = 0
            bin_itemids = list()

            for i in data['all_items']:
                if x[i, b].solution_value() > 0:

                    """ デバッグ用
                    print(
                        f"Item {i} 重量: {data['weights'][i]} 個数: {data['values'][i]}"
                    )
                    txt += f"  Item {i} 重量: {data['weights'][i]} 個数: {data['values'][i]}"
                    """

                    bin_weight += data['weights'][i]
                    bin_value += data['values'][i]
                    bin_itemids.append(i+1)

            """ デバッグ用
            print(f"  目的重量範囲は: { bin_minimumreq[b] } ~ {data['bin_capacities'][b]}")
            txt += f"  目的重量範囲は: { bin_minimumreq[b] } ~ {data['bin_capacities'][b]}"
            print(f'パック内の総重量は: {bin_weight}')
            txt += f'  パック内の総重量は: {bin_weight}'
            print(f'パック内の個数は: {bin_value}\n')
            txt += f'  パック内の個数は: {bin_value}\n'
            """

            resitem = {"asclass" : data["classes"][b], "new" : bin_weight, "items":bin_itemids}
            res.append(resitem)
            total_weight += bin_weight

        """ デバッグ用
        print(f'総重量は: {total_weight}') 
        txt += f'  総重量は: {total_weight}'
        with open('result.txt', 'w') as f:
            f.write(txt)
        """
        return res
    else:
        # print('最適解なし')
        return False

In [ ]:
def edakariA(bins, weights):   

    done = False
    while True:
        capacities = sorted(bins['min'])
        weights = sorted(weights)
    
        for i in range(0,len(capacities)):
            if capacities[i] < weights[i]:

                bins = bins[bins["min"]!=capacities[i]]
                break
            
        print(f'i={i}  cap = {len(capacities)}')
        
        if i == len(capacities) -1:
            done = True
            break
        
        if done:
            break

    return bins

In [ ]:
def  edakariC(bins, weights):
    
    """貪欲法による最大値を取得 => 枝狩りに用いる

    Returns:
        _type_: int
    """
    
    maxs = sorted(bins['max'], reverse = True)
    mins = sorted(bins['min'], reverse = True)
    weights = sorted(weights, reverse = True)

    for j in range(0,len(weights)):

        for i in range(0, len(maxs)):

            if mins[i] < 0:
                continue

            if maxs[i] >= weights[j]:

                maxs[i] = maxs[i] - weights[j]
                mins[i] = mins[i] - weights[j]

                break

    donyokucount = i

    return donyokucount

In [ ]:
def edakariB(bins, weights):

    while True:

        totalweights = sum(weights)
        totalmin = bins["min"].sum()

        if totalweights < totalmin:
            bins = bins[bins["min"]!=bins["min"].max()]
        else:
            break

    return bins
    

In [ ]:
import pandas as pd
bins_df_original = pd.read_csv('class_large.csv')
items_df = pd.read_csv('teg.csv')
weights = items_df['list'].to_list()
values = [ 1 for x in weights]
bins = flag_calculate(bins_df_original)
bins = edakariA(bins, weights)
bins = edakariB(bins, weights)
# donyokucount = edakariC(bins,weights)  #貪欲カウントを用いる場合、コメント解除

In [ ]:
#classの組み合わせ作成
import itertools

bins = bins.reset_index()

bins_max = len(bins)
res = False

#第1回実施
# bins_df = bins#貪欲カウントを用いる場合、コメント解除
# data, bin_minimumreq = make_data(bins_df,weights,values)#貪欲カウントを用いる場合、コメント解除
# res = slv_multiknap(data, bin_minimumreq)#貪欲カウントを用いる場合、コメント解除

# for i in range(donyokucount,bins_max): #貪欲カウントを用いる場合、コメント解除
for i in range(0,bins_max): #貪欲カウントを用いる場合、コメントアウト
    bins_combs = itertools.combinations(bins.index, len(bins) - i)
    num = 0
    for bins_comb in bins_combs:
        bins_df = bins.iloc[list(bins_comb)]

        data, bin_minimumreq = make_data(bins_df,weights,values)

        res = slv_multiknap(data, bin_minimumreq)
        
        if res:
            break
        else:
            num += 1
    if res:
        break
    else:
        pass
        # print(f'{i+1}回目実行 解無し 組み合わせ数　{num}個') #デバッグ用
# if res:
#     print(f'{i+1}回目で処理完了') #デバッグ用
    
# else:
#     print(f'条件を満たす結果が見つかりませんでした') #デバッグ用 

In [ ]:
res_bins = items_df.set_index('id')
res_bins.loc[:,'new'] = 0
res_bins.loc[:,'asclass'] = ""
for bin in res:
    for i in bin['items']:
        res_bins.loc[i,'asclass'] = bin['asclass']
        res_bins.loc[i,'new'] = bin['new']
res_bins = res_bins.reindex(columns=['list','asclass','new'])
res_bins.to_csv('out.csv')